# two approaches to get the key of max value of dictionary

In [23]:
my_dict ={"java":100, "python":112, "c":11}

In [24]:
value = max(my_dict.values())
index = list(my_dict.values()).index(value)
list(my_dict.keys())[index]

'python'

In [25]:
import operator
print(max(my_dict.items(), key=operator.itemgetter(1)))  # itemgetter(1) means compare by the 2 column => value of our dict
print(max(my_dict.items(), key=operator.itemgetter(1))[0])

('python', 112)
python


In [26]:
my_dict ={"2":100, "1":112, "0":11}
max(my_dict.items(), key=operator.itemgetter(0))

('2', 100)

# Automatically Generate a PDF and send it by Email

- Introduction : 

  - You work for a company that sells second hand cars. Management wants to get a summary of the amounts of vehicles that have been sold at the end of every month. The company already has a web service which serves sales data at the end of every month but management wants an email to be sent out with an attached PDF so that data is more easily readable.

- What you'll do : 

  - Write a script that summarizes and processes sales data into different categories
  - Generate a PDF using Python
  - Automatically send a PDF by email

# reports.py
generate PDF files

In [ ]:
#!/usr/bin/env python3

from reportlab.platypus import SimpleDocTemplate
from reportlab.platypus import Paragraph, Spacer, Table, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

# generate the pdf file 
# filename is the name we want to name it, e.g. report.pdf, could also be a path + filename => /dennis/document/report.pdf
# table data is our input
# table_data has to be like this data structure [[...], [...], [...], .....]
def generate(filename, title, additional_info, table_data):
    styles = getSampleStyleSheet()
    report = SimpleDocTemplate(filename)
    report_title = Paragraph(title, styles["h1"])
    report_info = Paragraph(additional_info, styles["BodyText"])
    table_style = [('GRID', (0,0), (-1,-1), 1, colors.black),
                ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
                ('ALIGN', (0,0), (-1,-1), 'CENTER')]
    report_table = Table(data=table_data, style=table_style, hAlign="LEFT")
    empty_line = Spacer(1,20)
    report.build([report_title, empty_line, report_info, empty_line, report_table])



# emials.py

send emails

In [ ]:
#!/usr/bin/env python3

import email.message
import mimetypes
import os.path
import smtplib

# generate the emails
def generate(sender, recipient, subject, body, attachment_path):
    # """Creates an email with an attachement."""
    # Basic Email formatting
    message = email.message.EmailMessage()
    message["From"] = sender
    message["To"] = recipient
    message["Subject"] = subject
    message.set_content(body)  # create the body part 

    # Process the attachment and add it to the email
    attachment_filename = os.path.basename(attachment_path)
    mime_type, _ = mimetypes.guess_type(attachment_path)
    mime_type, mime_subtype = mime_type.split('/', 1)

    with open(attachment_path, 'rb') as ap:
        message.add_attachment(ap.read(),
                              maintype=mime_type,
                              subtype=mime_subtype,
                              filename=attachment_filename)

    return message

# send email function
def send(message):
    # """Sends the message to the configured SMTP server."""
    mail_server = smtplib.SMTP('localhost')
    mail_server.send_message(message)
    mail_server.quit()



# example.py
test example

In [ ]:
#!/usr/bin/env python3

import emails
import os
import reports

table_data=[
  ['Name', 'Amount', 'Value'],
  ['elderberries', 10, 0.45],
  ['figs', 5, 3],
  ['apples', 4, 2.75],
  ['durians', 1, 25],
  ['bananas', 5, 1.99],
  ['cherries', 23, 5.80],
  ['grapes', 13, 2.48],
  ['kiwi', 4, 0.49]]
reports.generate("/tmp/report.pdf", "A Complete Inventory of My Fruit", "This is all my fruit.", table_data)

sender = "automation@example.com"
receiver = "{}@example.com".format(os.environ.get('USER'))
subject = "List of Fruits"
body = "Hi\n\nI'm sending an attachment with all my fruit."

message = emails.generate(sender, receiver, subject, body, "/tmp/report.pdf")
emails.send(message)



# cars.py 

real task start !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
#!/usr/bin/env python3

import json
import locale
import sys
import emails
import os
import reports

# to read the file in directory
def load_data(filename):
    # """Loads the contents of filename as a JSON file."""
    with open(filename) as json_file:
        data = json.load(json_file)
    return data


# change the format to have better visualization
def format_car(car):
    # """Given a car dictionary, returns a nicely formatted name."""
    return "{} {} ({})".format(car["car_make"], car["car_model"], car["car_year"])


# proccess the data to get the revenue, most sales model, most popular year information
def process_data(data):
    # """Analyzes the data, looking for maximums.

    # # Returns a list of lines that summarize the information.
    # """
    max_revenue = {"revenue": 0}
    car_most_sales = {}
    car_most_popular_year = {}
    
    # go through all the data to do the calculation
    for item in data:
        
        # Calculate the revenue generated by this model (price * total_sales)
        # We need to convert the price from "$1234.56" to 1234.56
        item_price = locale.atof(item["price"].strip("$"))
        item_revenue = item["total_sales"] * item_price
        if item_revenue > max_revenue["revenue"]:
            item["revenue"] = item_revenue
            max_revenue = item
        
        # get alle the model and total_scales infomation for the later use
        # TODO: also handle max sales
        if item['car']['car_model'] not in car_most_sales:
            car_most_sales[item['car']['car_model']] = item['total_sales']
        else:
            car_most_sales[item['car']['car_model']] += item['total_sales']

        # get alle the year and total_scales infomation for the later use
        # TODO: also handle most popular car_year
        if item['car']['car_year'] not in car_most_popular_year:
            car_most_popular_year[item['car']['car_year']] = item['total_sales']
        else:
            car_most_popular_year[item['car']['car_year']] += item['total_sales']

    # print out to see whether the result is expected
    print(car_most_sales)
    print(car_most_popular_year)

    # get the key of the max value out of dict
    # which means, get the most sales model, most popular year info
    import operator
    model = max(car_most_sales.items(), key=operator.itemgetter(1))
    year = max(car_most_popular_year.items(), key=operator.itemgetter(1))

    
    # put all revenue, most sales model, most popular year information together
    summary = [
    "The {} generated the most revenue: ${}".format(
      format_car(max_revenue["car"]), max_revenue["revenue"]),
    ]
    
    summary.append('The {} had the most sales: {}'.format(model[0], model[1]))
    summary.append("The most popular year was {} with {} sales.".format(year[0], year[1]))

    return summary


# transfer the dictionary data structure into table structure
def cars_dict_to_table(car_data):
    # """Turns the data in car_data into a list of lists."""
    
    table_data = [["ID", "Car", "Price", "Total Sales"]]
    for item in car_data:
        table_data.append([item["id"], format_car(item["car"]), item["price"], item["total_sales"]])
    return table_data


def main(argv):
    # """Process the JSON data and generate a full report out of it."""
    
    # load the data and proccess the data to get info
    data = load_data("car_sales.json")
    summary = process_data(data)
    print(summary)
    
    # transfer data into table structure
    car_table_data = cars_dict_to_table(data)

    # TODO: turn this into a PDF report
    # seperate the title in pdf line by line by using <br/>
    new_summary = '<br/>'.join(summary)
    reports.generate("/tmp/cars.pdf", new_summary, 
                     "This is all the information of sales summary.", car_table_data)
    
    # TODO: send the PDF report as an email attachment
    # seperate the title in pdf line by line by using <br/>
    new_summary = '\n'.join(summary)
    msg = emails.generate("automation@example.com", "student-04-ff6c41a5804e@example.com",
                          "Sales summary for last month", new_summary, "/tmp/cars.pdf")
    emails.send(msg)


if __name__ == "__main__":
    main(sys.argv)


# Grant required permissions to the file cars.py and open it using nano editor.

**sudo chmod o+wx ~/scripts/cars.py**

**nano ~/scripts/cars.py**

# "car_sales.json" 
look like


In [ ]:
[{"id":1,"car":{"car_make":"Ford","car_model":"Club Wagon","car_year":1997},"price":"$5179.39","total_sales":446},
{"id":2,"car":{"car_make":"Acura","car_model":"TL","car_year":2005},"price":"$14558.19","total_sales":589},
{"id":3,"car":{"car_make":"Volkswagen","car_model":"Jetta","car_year":2009},"price":"$14879.11","total_sales":825},
{"id":4,"car":{"car_make":"Chevrolet","car_model":"Uplander","car_year":2006},"price":"$17045.06","total_sales":689},
{"id":5,"car":{"car_make":"Plymouth","car_model":"Roadrunner","car_year":1969},"price":"$14770.44","total_sales":691},
{"id":6,"car":{"car_make":"GMC","car_model":"Safari","car_year":2000},"price":"$13390.83","total_sales":531},
{"id":7,"car":{"car_make":"Lamborghini","car_model":"Murciélago","car_year":2003},"price":"$7267.94","total_sales":374},
{"id":8,"car":{"car_make":"GMC","car_model":"3500","car_year":1999},"price":"$19292.10","total_sales":638},
{"id":9,"car":{"car_make":"Maybach","car_model":"62","car_year":2004},"price":"$11020.45","total_sales":945},
{"id":10,"car":{"car_make":"Chevrolet","car_model":"Cavalier","car_year":2001},"price":"$10708.87","total_sales":870},
{"id":11,"car":{"car_make":"Plymouth","car_model":"Acclaim","car_year":1993},"price":"$7734.63","total_sales":1158},
{"id":12,"car":{"car_make":"Buick","car_model":"Century","car_year":1991},"price":"$16905.08","total_sales":933},
{"id":13,"car":{"car_make":"Dodge","car_model":"Daytona","car_year":1984},"price":"$11540.36","total_sales":371},
{"id":14,"car":{"car_make":"Ford","car_model":"Econoline E350","car_year":1996},"price":"$19948.00","total_sales":1126},
{"id":15,"car":{"car_make":"Nissan","car_model":"Quest","car_year":2001},"price":"$12528.50","total_sales":536},
{"id":16,"car":{"car_make":"Toyota","car_model":"Yaris","car_year":2007},"price":"$10754.12","total_sales":573},
{"id":17,"car":{"car_make":"Maserati","car_model":"Spyder","car_year":2005},"price":"$18285.40","total_sales":224},
{"id":18,"car":{"car_make":"Ram","car_model":"Dakota","car_year":2011},"price":"$8792.39","total_sales":1135},
{"id":19,"car":{"car_make":"Mitsubishi","car_model":"Diamante","car_year":1995},"price":"$19173.71","total_sales":152},
{"id":20,"car":{"car_make":"Honda","car_model":"Accord","car_year":2001},"price":"$12123.05","total_sales":1063},
{"id":21,"car":{"car_make":"Mercury","car_model":"Sable","car_year":1991},"price":"$11226.09","total_sales":1118},
{"id":22,"car":{"car_make":"Toyota","car_model":"Previa","car_year":1991},"price":"$14361.62","total_sales":1048},
{"id":23,"car":{"car_make":"GMC","car_model":"Safari","car_year":1993},"price":"$10153.72","total_sales":354},
{"id":24,"car":{"car_make":"Suzuki","car_model":"SJ","car_year":1989},"price":"$14230.45","total_sales":1007},
{"id":25,"car":{"car_make":"Ford","car_model":"Ranger","car_year":1991},"price":"$14350.35","total_sales":299},
{"id":26,"car":{"car_make":"Suzuki","car_model":"Daewoo Magnus","car_year":2006},"price":"$9796.72","total_sales":408},
{"id":27,"car":{"car_make":"Chevrolet","car_model":"Impala","car_year":2005},"price":"$10738.89","total_sales":583},
{"id":28,"car":{"car_make":"Jeep","car_model":"Wrangler","car_year":2005},"price":"$17614.00","total_sales":1078},
{"id":29,"car":{"car_make":"Dodge","car_model":"Viper","car_year":2002},"price":"$11424.24","total_sales":342},
{"id":30,"car":{"car_make":"Acura","car_model":"RSX","car_year":2004},"price":"$5401.24","total_sales":167}]

which in short look like this

In [ ]:
{
    "id": 47,
    "car": {
            "car_make": "Lamborghini",
            "car_model": "Murciélago",
            "car_year": 2002
    },
    "price": "$13724.05",
    "total_sales": 149
}